In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableBranch
from langchain_core.prompts import ChatPromptTemplate

In [4]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
load_dotenv()
llm = ChatOpenAI(
    temperature=0.6,
    model="glm-4.6",
    openai_api_key=os.getenv("ZAI_API_KEY"),
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

In [ ]:
# 定义模拟子智能体处理器

def booking_handler(request:str) -> str:
    """模拟预订智能体处理请求"""
    print("\n‑‑‑ 委托给预订处理器‑‑‑")
    return f"预订处理器已处理请求：'{request}'。结果：模拟预订动作。"

def info_handle(request:str) -> str:
    """模拟信息智能体处理请求"""
    print("\n‑‑‑ 委托给信息处理器‑‑‑")
    return f"信息处理器已处理请求：'{request}'。结果：模拟信息检索。"

def unclear_handle(request:str) -> str:
    """处理无法委托的请求"""
    print("\n‑‑‑ 处理不明确的请求‑‑‑")
    return f"协调者无法委托请求'{request}'。请补充说明。"


# 定义协调者路由链
coordinator_router_prompt = ChatPromptTemplate.from_messages([
    ("system",
    """分析用户请求，判断应由哪个专属处理器处理。
    ‑ 若请求涉及预订机票或酒店，输出 'booker'。
    ‑ 其他一般信息问题，输出 'info'。
    ‑ 若请求不明确或不属于上述类别，输出 'unclear'。
    只输出一个词：'booker'、'info' 或 'unclear'。"""),
    ("user","{request}")
])

coordinator_router_chain = coordinator_router_prompt | llm | StrOutputParser()

# 定义委托的逻辑
""" 
lambda就是一个快速函数，内嵌在函数里，这样代码更简单，可以看作def的轻量版本
"""

branches = {
    "booker":RunnablePassthrough.assign(output=lambda x:booking_handler(x['request']['request'])),
    "info":RunnablePassthrough.assign(output=lambda x:info_handle(x['request']['request'])),
    "unclear":RunnablePassthrough.assign(output = lambda x:unclear_handle(x['request']['request'])),
}

""" 
RunnableBranch 是LangChain中的条件分支组件，
类似于编程中的 if-elif-else 语句。

"""

delegation_branch = RunnableBranch(
    (lambda x:x['decision'].strip() == 'booker',branches["booker"]),
    (lambda x:x['decision'].strip() == 'info',branches["info"]),
    branches["unclear"]
)

"""
这是一个管道式处理链，使用 | 操作符连接多个处理步骤。
第一步骤是数据准备，decision是路由器的决策，request是用户提出的请求传给下一个
第二部是分支路由，执行相对应的逻辑
第三部是结果提取，返回响应

"""

cordinator_agent = {
    "decision":coordinator_router_chain,
    "request":RunnablePassthrough()
} | delegation_branch | (lambda x:x['output'])





In [8]:
print("--预定请求示例")

request_a = "帮我订一张从北京到贵阳的机票"

result_a = cordinator_agent.invoke({"request":request_a})

print("最终结果 A：",result_a)

print("\n‑‑‑ 信息请求示例‑‑‑")
request_b = "意大利的首都是哪里？"
result_b = cordinator_agent.invoke({"request": request_b})
print("最终结果 B:",result_b)

print("\n‑‑‑ 不明确请求示例‑‑‑")
request_c = "讲讲量子物理。"
result_c = cordinator_agent.invoke({"request": request_c})
print("最终结果 C: ",result_c)

--预定请求示例

‑‑‑ 委托给预订处理器‑‑‑
最终结果 A： 预订处理器已处理请求：'帮我订一张从北京到贵阳的机票'。结果：模拟预订动作。

‑‑‑ 信息请求示例‑‑‑

‑‑‑ 委托给信息处理器‑‑‑
最终结果 B: 信息处理器已处理请求：'意大利的首都是哪里？'。结果：模拟信息检索。

‑‑‑ 不明确请求示例‑‑‑

‑‑‑ 委托给信息处理器‑‑‑
最终结果 C:  信息处理器已处理请求：'讲讲量子物理。'。结果：模拟信息检索。
